In [1]:
import math
import logging
import numpy as np


logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

class Activation(object):

    @staticmethod
    def relu_activation(x):
        return max(0, x)

    @staticmethod
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def tanh(x):
        return (1 - np.exp(x)) / (1 + np.exp(x))

    @staticmethod
    def softmax(x):
        x_new = [np.exp(i) for i in x]
        sum_x_new = sum(x_new)
        return [sum_x_new / (i) for i in x_new]

    @staticmethod
    def derivate_relu(x):
        if x > 0:
            return 1
        return 0

    @staticmethod
    def derivate_sigmoid(x):
        return (Activation.sigmoid(x)) * (1 - Activation.sigmoid(x))

    @staticmethod
    def derivate_tanh(x):
        return - np.exp(x) / (1 + np.exp(x)) ** 2


In [2]:
class LossFunction(object):

    @staticmethod
    def cross_entropy(Y_pred, Y_train):
        if Y_pred == 1:
            return -np.log(Y_train)
        else:
            return -np.log(1 - Y_train)

    @staticmethod
    def hinge_loss(Y_pred, Y_train):
        return np.max(0, 1 - Y_pred * Y_train)

    @staticmethod
    def L1_loss(Y_pred, Y_train):
        return np.sum(np.absolute(Y_pred - Y_train))

    @staticmethod
    def L2_loss(Y_pred, Y_train):
        return np.sum(np.power((Y_pred - Y_train), 2)) / len(Y_train)


In [3]:
class NeuralNetwork(object):

        def __init__(self, train_x, train_y, hidden_layer=1, hidden_neurons=3, bias = 1):
            self.train_x = train_x
            self.train_y = train_y
            
            self.hidden_layer = hidden_layer
            self.hidden_neurons = hidden_neurons
            self.input_nodes = np.shape(train_x)[0]
            self.output_nodes = np.shape(train_y)[0]
            self.grad_out = list()
            self.bias = bias

            self.loop_counter = self.hidden_layer + 1
            # seed for the fixed random values
            np.random.seed(3)
            self.W_in = np.random.normal(0.0, 0.1, (self.input_nodes, self.hidden_neurons))
            self.W_out = np.random.normal(0.0, 0.1, (self.hidden_neurons, self.output_nodes))
            self.bias_weight_Mat = np.random.normal(0.0, 0.1, (1,self.loop_counter))
            
            # special case to check, override bias_weight_Mat
            self.bias_weight_Mat = [0.35, 0.65]          
            print(self.W_in, self.W_out, "First and last weight mat")
            self.Weight_Mat = list()
            self.Weight_Mat.append(self.W_in)
            
            if self.loop_counter > 2:
                for i in range(1, self.loop_counter-1):
                    self.Wh_i = np.random.normal(0.0, 0.1, (self.hidden_neurons, self.hidden_neurons))
                    self.Weight_Mat.append(self.Wh_i)
            self.Weight_Mat.append(self.W_out)
             
                    
        def L2_loss(self, Y_pred, Y_train):
            return np.sum(np.power((Y_pred - Y_train), 2)) / len(Y_train)
        
        def sigmoid(self, x):
            return 1 / (1 + np.exp(-x))

        def forward_prop(self):
            weight = self.Weight_Mat
            # Layewise activation output
            self.lw_net = list()
            self.lw_act_out = list()
            
            for i in range(len(weight)):
                print((weight[i]), i)
                print((self.train_x), i)
                X_i = np.dot(np.array(self.train_x).T, np.array(weight[i])) + self.bias_weight_Mat[i]
                self.lw_net.append(X_i)
                A_i = self.sigmoid(X_i)
                self.lw_act_out.append(A_i)
                self.train_x = (A_i).T
            self.E_total = 0
            
            for i in range(len(A_i)):
                E = self.L2_loss(np.array(A_i[i]), np.array(self.train_y[i]))
                self.E_total += E
            print("lw", self.lw_net)
            print("lw_ac_out", self.lw_act_out)
            print("total", self.E_total)
                        
        
#       backpropagation
        def back_prop(self):
            self.loss = self.E_total
            for j in reversed(range(self.loop_counter)):
                for i in range(len(self.lw_act_out[j])):
            
                    loss_to_out = self.train_y[i] - self.lw_act_out[j][i]
                    out_to_net = self.lw_act_out[j][i]*(1-self.lw_act_out[j][i])
                    for k in range(len(self.lw_net[j])):
                        net_to_weight_k = self.lw_net[j][k]
                        self.grad_out.append(loss_to_out*out_to_net*net_to_weight_k)

            return self.grad_out
#             for i in reversed(range(self.loop_counter)):
#                 for j in range()
        
        
        

In [4]:
nn = NeuralNetwork([[0.5], [0.1]], [[0.99], [0.01]], 1, 3)
print(nn.forward_prop())
print(nn.back_prop())

[[ 0.17886285  0.04365099  0.00964975]
 [-0.18634927 -0.02773882 -0.0354759 ]] [[-0.00827415 -0.06270007]
 [-0.00438182 -0.0477218 ]
 [-0.13138648  0.08846224]] First and last weight mat
[[ 0.17886285  0.04365099  0.00964975]
 [-0.18634927 -0.02773882 -0.0354759 ]] 0
[[0.5], [0.1]] 0
[[-0.00827415 -0.06270007]
 [-0.00438182 -0.0477218 ]
 [-0.13138648  0.08846224]] 1
[[0.60367383]
 [0.59122979]
 [0.58692728]] 1
lw [array([[0.4207965 , 0.36905161, 0.35127728]]), array([[0.56530015, 0.63585596]])]
lw_ac_out [array([[0.60367383, 0.59122979, 0.58692728]]), array([[0.637678  , 0.65381609]])]
total 0.23715040802471377
None


IndexError: index 1 is out of bounds for axis 0 with size 1